In [4]:
import requests
import re
import json
import pandas as pd
import datetime
import numpy as np

import geopandas

import plotly
import plotly.express as px
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from dash import dash_table
from jupyter_dash import JupyterDash

pd.set_option('display.max_columns', None) 
pd.options.mode.chained_assignment = None

In [3]:
df_full = pd.read_csv('full_dataset.csv')

In [13]:
one_day = '2023-06-01'
df_one_day = df_full[df_full['extraction_date'] == one_day]
df_one_day

,id,internal_id,title_abbreviation,address,price,price_per_unit_area,size_with_square_meter,construction_year,levels,bedrooms,bathrooms,date,extraction_date,geo_lat,geo_lng,company_title,account_id,ad_group_id,url,dimos,perioxi,type,area,price_per_area,level_list,min_level,level_length,temp_no,temp_date_unit,days_passed
0,815053331,47143151,Διαμέρισμα 54 τ.μ.,Αθήνα (Λυκαβηττός),560.0,10,54 τ.μ.,1970.0,"['-1', '0']",1.0,1.0,πριν από 2 ημέρες,2023-06-01,37.979777,23.742884,NaN,2267566.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθήνα,Λυκαβηττός,Διαμέρισμα,54.0,10.370370,"[-1.0, 0.0]",0.0,2,2,ημ,2
1,815047933,47142491,Διαμέρισμα 45 τ.μ.,Αθήνα (Άνω Πατήσια),420.0,9,45 τ.μ.,1969.0,['1'],1.0,1.0,πριν από 2 ημέρες,2023-06-01,38.017481,23.735684,NaN,1407351.0,459093.0,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθήνα,Άνω Πατήσια,Διαμέρισμα,45.0,9.333333,[1.0],1.0,1,2,ημ,2
2,815017929,47140206,Διαμέρισμα 60 τ.μ.,Αθήνα (Κουντουριώτικα),780.0,13,60 τ.μ.,1975.0,['2'],1.0,1.0,πριν από 2 ημέρες,2023-06-01,37.992291,23.755579,NaN,110841.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθήνα,Κουντουριώτικα,Διαμέρισμα,60.0,13.000000,[2.0],2.0,1,2,ημ,2
3,814952950,47138240,Διαμέρισμα 55 τ.μ.,Αθήνα (Πολύγωνο),600.0,11,55 τ.μ.,1981.0,['1'],1.0,1.0,πριν από 3 ημέρες,2023-06-01,37.999083,23.755417,NaN,1344554.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθήνα,Πολύγωνο,Διαμέρισμα,55.0,10.909091,[1.0],1.0,1,3,ημ,3
4,814956607,47138448,Διαμέρισμα 50 τ.μ.,Αθήνα (Πανόρμου),790.0,16,50 τ.μ.,1970.0,['1'],1.0,1.0,πριν από 3 ημέρες,2023-06-01,37.990937,23.765072,BLUEGROUND REAL ESTATE,1220409.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθήνα,Πανόρμου,Διαμέρισμα,50.0,15.800000,[1.0],1.0,1,3,ημ,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5451,798078305,46276831,Διαμέρισμα 150 τ.μ.,Βύρωνας,1050.0,7,150 τ.μ.,1974.0,['3'],3.0,1.0,πριν από 2 μήνες,2023-06-01,37.955561,23.750699,Homenet Real Estate & Development,1161699.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Βύρωνας,NaN,Διαμέρισμα,150.0,7.000000,[3.0],3.0,1,2,μή,60
5452,799256815,46334139,Διαμέρισμα 150 τ.μ.,Ζωγράφου (Κέντρο),1000.0,7,150 τ.μ.,NaN,['3'],3.0,1.0,πριν από 2 μήνες,2023-06-01,37.976858,23.770422,Dimit Estate,1144164.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Ζωγράφου,Κέντρο,Διαμέρισμα,150.0,6.666667,[3.0],3.0,1,2,μή,60
5453,804971100,46592632,Μονοκατοικία 600 τ.μ.,Φιλοθέη,8000.0,13,600 τ.μ.,NaN,['0'],4.0,3.0,πριν από 2 μήνες,2023-06-01,38.023565,23.777302,Moisiadis Real Estate,451111.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Φιλοθέη,NaN,Μονοκατοικία,600.0,13.333333,[0.0],0.0,1,2,μή,60
5454,2550572,21407274,Διαμέρισμα 50 τ.μ.,Βύρωνας (Αγορά),590.0,12,50 τ.μ.,1970.0,['1'],2.0,NaN,πριν από 2 μήνες,2023-06-01,37.959260,23.756369,NaN,88626.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Βύρωνας,Αγορά,Διαμέρισμα,50.0,11.800000,[1.0],1.0,1,2,μή,60


In [14]:
df_one_day.groupby('dimos')[['price_per_unit_area']].mean().sort_values('price_per_unit_area', ascending=False)

,price_per_unit_area
dimos,
Γλυφάδα,18.000000
Ψυχικό,14.676056
Μαρούσι,14.111111
Φιλοθέη,13.208333
Νέο Ψυχικό,11.687500
Καισαριανή,11.012048
Χαλάνδρι,10.739130
Άγιος Δημήτριος,10.500000
Δάφνη,10.487805


In [15]:
df_one_day.groupby('dimos').aggregate({'price_per_unit_area': ['mean', 'count', 'min', 'max'], 'construction_year': ['mean', 'min', 'max']}).reset_index().sort_values(by=('price_per_unit_area', 'mean'), ascending=False)

dimos price_per_unit_area               construction_year  \
                                   mean count min max              mean   
8           Γλυφάδα           18.000000     1  18  18       2011.000000   
27           Ψυχικό           14.676056    71   7  25       1980.786885   
14          Μαρούσι           14.111111    18   8  20       1981.166667   
24          Φιλοθέη           13.208333    48   6  22       1984.184211   
19       Νέο Ψυχικό           11.687500    48   5  22       1987.902439   
12       Καισαριανή           11.012048    83   6  22       1980.656716   
25         Χαλάνδρι           10.739130    23   8  16       1983.500000   
1   Άγιος Δημήτριος           10.500000     2  10  11       1980.000000   
9             Δάφνη           10.487805    41   5  25       1988.861111   
4             Αθήνα           10.344023  3898   4  25       1973.860428   
15          Μοσχάτο           10.222222     9   6  18       1973.375000   
23          Υμηττός            9.954545    22   6  16       1983.222222   
10         Ζωγράφου            9.869565   391   4  24       1978.624595   
6           Βύρωνας            9.551724   116   5  19       1983.846939   
13         Καλλιθέα            9.542857   280   5  22       1979.128440   
22           Ταύρος            9.411765    17   6  13       1989.375000   
20          Παπάγου            9.333333     3   9  10       1975.000000   
17       Νέα Σμύρνη            9.250000    72   6  21       1977.258621   
16        Νέα Ιωνία            8.714286    14   6  12       1984.555556   
7           Γαλάτσι            8.500000    94   6  17       1985.909091   
21        Περιστέρι            8.070175   114   4  15       1987.921053   
26         Χολαργός            8.000000     1   8   8               NaN   
2    Άγιος Νικόλαος            8.000000     1   8   8               NaN   
5           Αιγάλεω            7.555556     9   5  12       1980.666667   
3             Ίλιον            7.536585    41   4  14       1991.181818   
11        Ηλιούπολη            7.500000     2   7   8       1988.000000   
18    Νέα Χαλκηδόνα            7.266667    15   5  12       1977.076923   
0   Άγιοι Ανάργυροι            7.227273    22   5  12       1985.769231   

                    
       min     max  
8   2011.0  2011.0  
27  1948.0  2020.0  
14  1960.0  2001.0  
24  1952.0  2023.0  
19  1960.0  2023.0  
12  1958.0  2017.0  
25  1968.0  2005.0  
1   1980.0  1980.0  
9   1964.0  2022.0  
4   1900.0  2023.0  
15  1956.0  1990.0  
23  1954.0  2012.0  
10  1958.0  2023.0  
6   1950.0  2023.0  
13  1952.0  2023.0  
22  1964.0  2022.0  
20  1970.0  1985.0  
17  1945.0  2019.0  
16  1970.0  2008.0  
7   1970.0  2023.0  
21  1950.0  2023.0  
26     NaN     NaN  
2      NaN     NaN  
5   1970.0  1990.0  
3   1950.0  2023.0  
11  1978.0  1998.0  
18  1957.0  2018.0  
0   1960.0  2017.0

In [16]:
df_one_day.groupby('perioxi').aggregate({'price_per_unit_area': ['mean', 'count', 'min', 'max'], 'construction_year': ['mean', 'min', 'max']}).reset_index().sort_values(by=('price_per_unit_area', 'mean'), ascending=False)

perioxi price_per_unit_area                \
                                           mean count min max   
136                Ρηγίλλης           20.444444     9  13  24   
118                   Πλάκα           16.166667    12  12  22   
69                 Κολωνάκι           16.110048   209   5  25   
92              Μοναστηράκι           15.666667     9   8  20   
52                  Ζάππειο           15.650000    20   9  23   
..                      ...                 ...   ...  ..  ..   
37                   Βιαμάξ            6.428571     7   6   7   
152                   Φλέβα            6.000000     1   6   6   
6    Άγιος Ιωάννης Θεολόγος            6.000000     7   4   8   
150                 Τσούμπα            5.500000     2   5   6   
81        Λόφος Αξιωματικών            5.500000     2   5   6   

    construction_year                  
                 mean     min     max  
136       1965.333333  1947.0  2023.0  
118       1952.750000  1900.0  1970.0  
69        1967.011299  1930.0  2010.0  
92        1957.666667  1901.0  2018.0  
52        1970.941176  1933.0  2001.0  
..                ...     ...     ...  
37        1999.000000  1978.0  2005.0  
152               NaN     NaN     NaN  
6         1980.000000  1977.0  1985.0  
150       1965.000000  1960.0  1970.0  
81        1998.000000  1998.0  1998.0  

[158 rows x 8 columns]

In [17]:
df_one_day[df_one_day['dimos'] == "Αθήνα"]['perioxi'].value_counts().to_dict()

{'Παγκράτι': 218,
 'Κολωνάκι': 209,
 'Κυψέλη': 205,
 'Άνω Κυψέλη': 129,
 'Αμπελόκηποι': 123,
 'Πλατεία Αμερικής': 104,
 'Πλατεία Αττικής': 94,
 'Άγιος Παντελεήμονας': 94,
 'Άνω Πατήσια': 87,
 'Πανόρμου': 86,
 'Ιπποκράτους': 86,
 'Κουντουριώτικα': 83,
 'Γκύζη': 82,
 'Κολιάτσου': 80,
 'Νέα Κυψέλη': 74,
 'Λυκαβηττός': 72,
 'Χίλτον': 67,
 'Μουσείο': 67,
 'Εξάρχεια': 65,
 'Άγιος Νικόλαος': 63,
 'Πεδίον Άρεως': 61,
 'Προφήτης Ηλίας': 61,
 'Ακαδημία Πλάτωνος': 61,
 'Κουκάκι': 60,
 'Ιπποκράτειο': 59,
 'Πλατεία Βικτωρίας': 58,
 'Μεταξουργείο': 52,
 'Πολύγωνο': 51,
 'Κολωνός': 49,
 'Ελληνορώσων': 48,
 'Ηπείρου': 48,
 'Νεάπολη': 46,
 'Ιλίσια': 43,
 'Κάτω Πετράλωνα': 42,
 'Άγιος Ελευθέριος': 41,
 'Βαρνάβα': 40,
 'Άγιος Λουκάς': 38,
 'Νέος Κόσμος': 38,
 'Άγιος Θωμάς': 35,
 'Ερυθρός': 30,
 'Άγιος Αρτέμιος': 30,
 'Κάτω Πατήσια': 30,
 'Άνω Πετράλωνα': 29,
 'Λόφος Στρέφη': 29,
 'Σταθμός Λαρίσης': 27,
 'Αγία Παρασκευή': 27,
 'Νιρβάνα': 25,
 'Καλλιρρόης': 24,
 'Νοσοκομείο Παίδων': 24,
 'Πλατεία Βάθης': 2

In [53]:
url_count = df_full.groupby('id')[['url']].nunique()
url_count[url_count['url'] > 1]

,url
id,
576566,2
2465980,2
62783106,2
482987552,2
631954687,2
...,...
815340607,2
815358102,2
815360756,2


In [54]:
df_full.loc[df_full['id'] == 576566, ['id', 'dimos', 'perioxi', 'price', 'area', 'url']]

,id,dimos,perioxi,price,area,url
5446,576566,Βύρωνας,NaN,579.0,50.0,https://www.xe.gr/property/d/enoikiaseis-katoi...
10881,576566,Βύρωνας,NaN,579.0,50.0,https://www.xe.gr/property/d/enoikiaseis-katoi...
16319,576566,Βύρωνας,NaN,579.0,50.0,https://www.xe.gr/property/d/enoikiaseis-katoi...
21006,576566,Βύρωνας,NaN,579.0,50.0,https://www.xe.gr/property/d/enoikiaseis-katoi...
26812,576566,Βύρωνας,Αγορά,579.0,50.0,https://www.xe.gr/property/d/enoikiaseis-katoi...


In [55]:
a = df_full.loc[df_full['id'] == 576566, 'url']
for b in a:
    print(b)

https://www.xe.gr/property/d/enoikiaseis-katoikion/576566/vyrwnas-579-50
https://www.xe.gr/property/d/enoikiaseis-katoikion/576566/vyrwnas-579-50
https://www.xe.gr/property/d/enoikiaseis-katoikion/576566/vyrwnas-579-50
https://www.xe.gr/property/d/enoikiaseis-katoikion/576566/vyrwnas-579-50
https://www.xe.gr/property/d/enoikiaseis-katoikion/576566/vyrwnas-agora-579-50


In [3]:
df_full['MaxDateSeen'] = df_full.groupby('id')['extraction_date'].transform('max')

In [4]:
df_full[df_full['MaxDateSeen'] < "2023-06-05"]

,id,internal_id,title_abbreviation,address,price,price_per_unit_area,size_with_square_meter,construction_year,levels,bedrooms,bathrooms,date,extraction_date,geo_lat,geo_lng,company_title,account_id,ad_group_id,url,dimos,perioxi,type,area,price_per_area,level_list,min_level,level_length,temp_no,temp_date_unit,days_passed,MaxDateSeen
8,814547213,47119901,Διαμέρισμα 34 τ.μ.,Αθήνα (Ιπποκράτειο),450.0,13,34 τ.μ.,NaN,['5'],1.0,1.0,πριν από 1 εβδομάδα,2023-06-01,37.983694,23.760134,NaN,1416328.0,458953.0,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθήνα,Ιπποκράτειο,Διαμέρισμα,34.0,13.235294,[5.0],5.0,1,1,εβ,7,2023-06-01
12,814509653,47118980,Μονοκατοικία 45 τ.μ.,Αθήνα (Ελαιώνας),320.0,7,45 τ.μ.,1970.0,['0'],1.0,2.0,πριν από 1 εβδομάδα,2023-06-01,37.990350,23.696610,One Real Estate,1168303.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθήνα,Ελαιώνας,Μονοκατοικία,45.0,7.111111,[0.0],0.0,1,1,εβ,7,2023-06-01
13,814981443,47138678,Διαμέρισμα 23 τ.μ.,Αθήνα (Άνω Πατήσια),270.0,12,23 τ.μ.,NaN,['4'],1.0,1.0,πριν από 2 ημέρες,2023-06-01,38.016450,23.732062,NaN,795832.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθήνα,Άνω Πατήσια,Διαμέρισμα,23.0,11.739130,[4.0],4.0,1,2,ημ,2,2023-06-03
22,814396726,47113917,Διαμέρισμα 38 τ.μ.,Αθήνα (Ηπείρου),350.0,9,38 τ.μ.,1970.0,['3'],1.0,1.0,πριν από 1 εβδομάδα,2023-06-01,37.991510,23.724500,One Real Estate,1168303.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθήνα,Ηπείρου,Διαμέρισμα,38.0,9.210526,[3.0],3.0,1,1,εβ,7,2023-06-04
23,814822124,47132891,Διαμέρισμα 65 τ.μ.,Αθήνα (Ακαδημία Πλάτωνος),450.0,7,65 τ.μ.,1970.0,['1'],1.0,NaN,πριν από 4 ημέρες,2023-06-01,37.994711,23.706262,Energy Real Estate,2203409.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθήνα,Ακαδημία Πλάτωνος,Διαμέρισμα,65.0,6.923077,[1.0],1.0,1,4,ημ,4,2023-06-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21661,804938667,46591752,Διαμέρισμα 100 τ.μ.,Καισαριανή (Νήαρ Ηστ),800.0,8,100 τ.μ.,1980.0,['1'],2.0,1.0,πριν από 2 μήνες,2023-06-04,37.969422,23.758668,Equal Estate,2042950.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Καισαριανή,Νήαρ Ηστ,Διαμέρισμα,100.0,8.000000,[1.0],1.0,1,2,μή,60,2023-06-04
21706,806632212,46692695,Διαμέρισμα 97 τ.μ.,Ηλιούπολη (Λόφος Γερμανού),750.0,8,97 τ.μ.,1978.0,['1'],2.0,1.0,πριν από 2 μήνες,2023-06-04,37.948925,23.753605,Τσαούσης - Σταυρόπουλος,215073.0,403713.0,https://www.xe.gr/property/d/enoikiaseis-katoi...,Ηλιούπολη,Λόφος Γερμανού,Διαμέρισμα,97.0,7.731959,[1.0],1.0,1,2,μή,60,2023-06-04
21710,808982851,46823195,Διαμέρισμα 80 τ.μ.,Καλλιθέα (Λόφος Σικελίας),750.0,9,80 τ.μ.,1981.0,['5'],1.0,1.0,πριν από 2 μήνες,2023-06-04,37.961959,23.714428,Helen's Real Estate,408387.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Καλλιθέα,Λόφος Σικελίας,Διαμέρισμα,80.0,9.375000,[5.0],5.0,1,2,μή,60,2023-06-04
21795,803694238,46530753,Διαμέρισμα 90 τ.μ.,Μενεμένη (Αμπελόκηποι),800.0,9,90 τ.μ.,1974.0,['4'],1.0,1.0,πριν από 2 μήνες,2023-06-04,37.989917,23.760045,Equal Estate,2042950.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθήνα,Αμπελόκηποι,Διαμέρισμα,90.0,8.888889,[4.0],4.0,1,2,μή,60,2023-06-04


In [72]:
df_full.groupby('MaxDateSeen').size()

MaxDateSeen
2023-06-01      272
2023-06-02      342
2023-06-03      302
2023-06-04      717
2023-06-05    25630
dtype: int64

In [33]:
app = JupyterDash(__name__)

In [34]:
df = df_full
app.layout = html.Div([
    html.H1("Interactive Dashboard"),
    html.Label("Filter 1:"),
    dcc.Dropdown(
        id='filter1',
        options=[
            {'label': column, 'value': column}
            for column in df.columns
        ],
        value=None
    ),
    html.Label("Filter 2:"),
    dcc.Dropdown(
        id='filter2',
        options=[
            {'label': column, 'value': column}
            for column in df.columns
        ],
        value=None
    ),
    html.Br(),
    dash_table.DataTable(
        id='table',
        columns=[{'name': col, 'id': col} for col in df.columns],
        style_table={'overflowX': 'auto'},
        style_cell={
            'minWidth': '0px', 'maxWidth': '180px',
            'whiteSpace': 'normal',
            'textAlign': 'left',
        },
    ),
])

In [38]:
@app.callback(
    Output('table', 'data'),
    [Input('filter1', 'value'),
     Input('filter2', 'value')]
)
def update_table(filter1_value, filter2_value):
    filtered_df = df
    cols_to_keep = [x  for x in [filter1_value, filter2_value] if x]
    
    return dash_table.DataTable(df_full[cols_to_keep].to_dict('records'))

In [39]:
app.run_server(mode='inline')

Dash is running on http://127.0.0.1:8050/



In [46]:
app = JupyterDash(__name__)
df = df_full[['type', 'dimos', 'price', 'area', 'bedrooms', 'bathrooms', 'min_level', 'extraction_date']]

app.layout = html.Div([
    html.H1("Interactive Dashboard"),
    html.Label("Filter 1:"),
    dcc.Dropdown(
        id='filter1',
        options=[
            {'label': str(date), 'value': date}
            for date in df['extraction_date'].unique()
        ],
        value=None
    ),
    html.Br(),
    html.Label("Dimos Filter:"),
    dcc.Dropdown(
        id='dimos-filter',
        options=[
            {'label': str(dimos), 'value': dimos}
            for dimos in df['dimos'].unique()
        ],
        value=None
    ),
    html.Br(),
    dash_table.DataTable(
        id='table',
        columns=[{'name': col, 'id': col} for col in df.columns],
        style_table={'overflowX': 'auto'},
        style_cell={
            'minWidth': '0px', 'maxWidth': '180px',
            'whiteSpace': 'normal',
            'textAlign': 'left',
        },
    ),
])

@app.callback(
    Output('table', 'data'),
    [Input('filter1', 'value'), 
     Input('dimos-filter', 'value')]
)
def update_table(filter1_value, dimos_filtered):
    if filter1_value is None:
        filtered_df = df
    else:
        filtered_df = df[df['extraction_date'] == filter1_value]
        
    if dimos_filtered:
        filtered_df = filtered_df[filtered_df['dimos'] == dimos_filtered]
    
    return filtered_df.head(50).to_dict('records')

app.run_server(mode='inline')

Dash is running on http://127.0.0.1:8050/



In [57]:
# app = JupyterDash(__name__, external_stylesheets=['styles_test.css'])
# app = JupyterDash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])
app = JupyterDash(__name__)
df = df_full[['type', 'dimos', 'price', 'area', 'bedrooms', 'bathrooms', 'min_level', 'extraction_date']]


app.layout = html.Div(
    className="container",
    children=[
        html.H1("Interactive Dashboard"),
        html.Div(
            className="sidebar",
            children=[
                html.Label("Filter 1:"),
                dcc.Dropdown(
                    id='filter1',
                    options=[
                        {'label': str(date), 'value': date}
                        for date in df['extraction_date'].unique()
                    ],
                    value=None
                ),
                html.Label("Δήμος!"),
                dcc.Dropdown(
                    id='dimos-filter',
                    options=[
                        {'label': dimos, 'value': dimos}
                        for dimos in df['dimos'].unique()
                    ],
                    value=None
                ),
            ],
        ),
        html.Div(
            className="content",
            children=[
                dash_table.DataTable(
                    id='table',
                    columns=[{'name': col, 'id': col} for col in df.columns],
                    style_table={'overflowX': 'auto'},
                    style_cell={
                        'minWidth': '0px', 'maxWidth': '180px',
                        'whiteSpace': 'normal',
                        'textAlign': 'left',
                    },
                ),
            ],
        ),
    ],
)


@app.callback(
    Output('table', 'data'),
    [Input('filter1', 'value'), 
     Input('dimos-filter', 'value')]
)
def update_table(filter1_value, dimos_filtered):
    if filter1_value is None:
        filtered_df = df
    else:
        filtered_df = df[df['extraction_date'] == filter1_value]
        
    if dimos_filtered:
        filtered_df = filtered_df[filtered_df['dimos'] == dimos_filtered]
    
    return filtered_df.head(50).to_dict('records')

app.run_server(mode='inline')

Dash is running on http://127.0.0.1:8050/



In [60]:
# app = JupyterDash(__name__, external_stylesheets=['styles_test.css'])
# app = JupyterDash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])
app = JupyterDash(__name__)
df = df_full[['type', 'dimos', 'price', 'area', 'bedrooms', 'bathrooms', 'min_level', 'extraction_date']]


app.layout = html.Div(
    className="container",
    children=[
        html.H1("Interactive Dashboard"),
        html.Div(
            style={
                "display": "inline-block",
                "verticalAlign": "top",
                "width": "20%",
                "padding": "10px",
            },
            children=[
                html.Label("Filter 1:"),
                dcc.Dropdown(
                    id='filter1',
                    options=[
                        {'label': str(date), 'value': date}
                        for date in df['extraction_date'].unique()
                    ],
                    value=None
                ),
                html.Label("Δήμος!"),
                dcc.Dropdown(
                    id='dimos-filter',
                    options=[
                        {'label': dimos, 'value': dimos}
                        for dimos in df['dimos'].unique()
                    ],
                    value=None
                ),
            ],
        ),
        html.Div(
            style={
                "display": "inline-block",
                "width": "70%",
                "padding": "10px",
            },
            children=[
                dash_table.DataTable(
                    id='table',
                    columns=[{'name': col, 'id': col} for col in df.columns],
                    style_table={'overflowX': 'auto'},
                    style_cell={
                        'minWidth': '0px', 'maxWidth': '180px',
                        'whiteSpace': 'normal',
                        'textAlign': 'left',
                        'userSelect': 'text',
                    },
                ),
            ],
        ),
    ],
)


@app.callback(
    Output('table', 'data'),
    [Input('filter1', 'value'), 
     Input('dimos-filter', 'value')]
)
def update_table(filter1_value, dimos_filtered):
    if filter1_value is None:
        filtered_df = df
    else:
        filtered_df = df[df['extraction_date'] == filter1_value]
        
    if dimos_filtered:
        filtered_df = filtered_df[filtered_df['dimos'] == dimos_filtered]
    
    return filtered_df.head(50).to_dict('records')

app.run_server(mode='inline')

Dash is running on http://127.0.0.1:8050/



In [61]:
df_full[]

,id,internal_id,title_abbreviation,address,price,price_per_unit_area,size_with_square_meter,construction_year,levels,bedrooms,bathrooms,date,extraction_date,geo_lat,geo_lng,company_title,account_id,ad_group_id,url,dimos,perioxi,type,area,price_per_area,level_list,min_level,level_length,temp_no,temp_date_unit,days_passed,MaxDateSeen
0,815053331,47143151,Διαμέρισμα 54 τ.μ.,Αθήνα (Λυκαβηττός),560.0,10,54 τ.μ.,1970.0,"['-1', '0']",1.0,1.0,πριν από 2 ημέρες,2023-06-01,37.979777,23.742884,NaN,2267566.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθήνα,Λυκαβηττός,Διαμέρισμα,54.0,10.370370,"[-1.0, 0.0]",0.0,2,2,ημ,2,2023-06-05
1,815047933,47142491,Διαμέρισμα 45 τ.μ.,Αθήνα (Άνω Πατήσια),420.0,9,45 τ.μ.,1969.0,['1'],1.0,1.0,πριν από 2 ημέρες,2023-06-01,38.017481,23.735684,NaN,1407351.0,459093.0,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθήνα,Άνω Πατήσια,Διαμέρισμα,45.0,9.333333,[1.0],1.0,1,2,ημ,2,2023-06-05
2,815017929,47140206,Διαμέρισμα 60 τ.μ.,Αθήνα (Κουντουριώτικα),780.0,13,60 τ.μ.,1975.0,['2'],1.0,1.0,πριν από 2 ημέρες,2023-06-01,37.992291,23.755579,NaN,110841.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθήνα,Κουντουριώτικα,Διαμέρισμα,60.0,13.000000,[2.0],2.0,1,2,ημ,2,2023-06-05
3,814952950,47138240,Διαμέρισμα 55 τ.μ.,Αθήνα (Πολύγωνο),600.0,11,55 τ.μ.,1981.0,['1'],1.0,1.0,πριν από 3 ημέρες,2023-06-01,37.999083,23.755417,NaN,1344554.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθήνα,Πολύγωνο,Διαμέρισμα,55.0,10.909091,[1.0],1.0,1,3,ημ,3,2023-06-05
4,814956607,47138448,Διαμέρισμα 50 τ.μ.,Αθήνα (Πανόρμου),790.0,16,50 τ.μ.,1970.0,['1'],1.0,1.0,πριν από 3 ημέρες,2023-06-01,37.990937,23.765072,BLUEGROUND REAL ESTATE,1220409.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθήνα,Πανόρμου,Διαμέρισμα,50.0,15.800000,[1.0],1.0,1,3,ημ,3,2023-06-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27258,632024644,41302418,Διαμέρισμα 64 τ.μ.,Βύρωνας,640.0,10,64 τ.μ.,NaN,['2'],2.0,NaN,πριν από 2 μήνες,2023-06-05,37.959404,23.750014,NaN,98213.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Βύρωνας,NaN,Διαμέρισμα,64.0,10.000000,[2.0],2.0,1,2,μή,60,2023-06-05
27259,798078305,46276831,Διαμέρισμα 150 τ.μ.,Βύρωνας,1050.0,7,150 τ.μ.,1974.0,['3'],3.0,1.0,πριν από 2 μήνες,2023-06-05,37.955561,23.750699,Homenet Real Estate & Development,1161699.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Βύρωνας,NaN,Διαμέρισμα,150.0,7.000000,[3.0],3.0,1,2,μή,60,2023-06-05
27260,799256815,46334139,Διαμέρισμα 150 τ.μ.,Ζωγράφου (Κέντρο),1000.0,7,150 τ.μ.,NaN,['3'],3.0,1.0,πριν από 2 μήνες,2023-06-05,37.976858,23.770422,Dimit Estate,1144164.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Ζωγράφου,Κέντρο,Διαμέρισμα,150.0,6.666667,[3.0],3.0,1,2,μή,60,2023-06-05
27261,2465980,21365332,Διαμέρισμα 50 τ.μ.,Βύρωνας (Αγορά),590.0,12,50 τ.μ.,1954.0,['1'],1.0,NaN,πριν από 2 μήνες,2023-06-05,37.959260,23.756369,NaN,98222.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Βύρωνας,Αγορά,Διαμέρισμα,50.0,11.800000,[1.0],1.0,1,2,μή,60,2023-06-05


In [13]:
fig = px.bar(df_full[df_full['extraction_date'] == "2023-06-05"].groupby('dimos')['price_per_area'].mean().reset_index(), x='dimos', y='price_per_area')
fig.show()

In [12]:
fig = px.bar(df_full[df_full['extraction_date'] == "2023-06-05"].groupby('min_level')['price_per_area'].mean().reset_index(), x='min_level', y='price_per_area')
fig.show()

In [ ]:
df_full[df_full['extraction_date'] == "2023-06-05"].groupby('min_level')['price_per_area'].mean().reset_index()

In [15]:
df_full["is_new"] = df_full["days_passed"] == 0

In [68]:
# df_test = df_full.copy()
df_test = df_full.groupby('extraction_date')

In [29]:
fig = px.bar(df_full.groupby('extraction_date')['is_new'].sum().reset_index(), x='is_new', y='extraction_date', 
             orientation='h', title='New ads')
fig.update_traces(marker_color='darkcyan')
fig.update_layout(
    xaxis_title='New ads',
    yaxis_title='Date'
)

fig.show()

In [67]:
fig = px.line(df_full.groupby('extraction_date')['is_new'].sum().reset_index(), 
              x='extraction_date', y='is_new', title='New ads')
fig.update_traces(line_color='darkcyan')
fig.update_layout(
    xaxis=dict(tickmode='linear', dtick='D1', tickformat='%d/%m/%Y'),
#     yaxis=dict(showgrid=False, zeroline=False),
    xaxis_title='Date',
    yaxis_title='New ads'
)


fig.show()

In [62]:
df_ad_age = df_full[df_full['extraction_date'] == "2023-06-05"].groupby('days_passed').size().reset_index(name='Count')
df_ad_age['age'] = pd.cut(df_ad_age['days_passed'], bins=[-1, 0, 7, 14, 21, 28, 60], labels=['1 day', '< 1 week', '< 2 weeks', '< 3 weeks', '< 1 month', '< 2 months'])
df_age_graph = df_ad_age.groupby('age')['Count'].sum().reset_index(name='Count')
fig = px.bar(df_age_graph, x='age', y='Count', 
             orientation='v', title='Ad age (2023-06-05)')
fig.update_traces(marker_color='darkcyan')
fig.update_traces(texttemplate='%{text:.2s}', textposition='inside')
fig.update_layout(
    xaxis_title='Age',
    yaxis_title='Count'
)

fig.show()

In [51]:
df_full[df_full['dimos'].isin(['Μοσχάτο', 'Ταύρος'])]['id'].nunique()

27

In [64]:
df = px.data.gapminder().query("continent == 'Europe' and year == 2007 and pop > 2.e6")
fig = px.bar(df, y='pop', x='country', text='pop')
fig.update_traces(texttemplate='%{text:.2s}', textposition='inside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()

In [70]:
df_full['id'].nunique()

6206

In [73]:
df2 = pd.read_csv('full_dataset2.csv')
df_ypes = pd.read_csv('kwd_ypes.csv')

In [74]:
df_ypes

,dimos,kwd_ypes,enotites
0,Άγιοι Ανάργυροι - Καματερό,9180,ΔΥΤΙΚΟΥ ΤΟΜΕΑ ΑΘΗΝΩΝ
1,Άγιος Δημήτριος,9194,ΝΟΤΙΟΣ ΤΟΜΕΑΣ ΑΘΗΝΩΝ
2,Αθηναίων,9186,ΚΕΝΤΡΙΚΟΣ ΤΟΜΕΑΣ ΑΘΗΝΩΝ
3,Αιγάλεω,9181,ΔΥΤΙΚΟΥ ΤΟΜΕΑ ΑΘΗΝΩΝ
4,Βύρωνας,9187,ΚΕΝΤΡΙΚΟΣ ΤΟΜΕΑΣ ΑΘΗΝΩΝ
5,Γαλάτσι,9188,ΚΕΝΤΡΙΚΟΣ ΤΟΜΕΑΣ ΑΘΗΝΩΝ
6,Γλυφάδα,9196,ΝΟΤΙΟΣ ΤΟΜΕΑΣ ΑΘΗΝΩΝ
7,Δάφνη - Υμηττός,9189,ΚΕΝΤΡΙΚΟΣ ΤΟΜΕΑΣ ΑΘΗΝΩΝ
8,Ζωγράφου,9190,ΚΕΝΤΡΙΚΟΣ ΤΟΜΕΑΣ ΑΘΗΝΩΝ
9,Ηλιούπολη,9191,ΚΕΝΤΡΙΚΟΣ ΤΟΜΕΑΣ ΑΘΗΝΩΝ


In [75]:
df3 = df2.merge(df_ypes, on='dimos')

In [79]:
df3.to_csv('full_dataset2_with_types.csv', encoding='utf-8-sig', index=False)